<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Regresión

Ejemplo de regresión lineal<br>
v2.0

### Objetivos: 
Generar un modelo base sin utilizar Inteligencia Artificial para calcular el precio promedio de un alquiler para compararlo con el resultado del modelo de la Regresión Lineal.
*   Preprocesar los datos (descarga, lectura, limplieza y filtrado)
*  Conocer como funciona la regresión lineal empleando un dataset de propiedades en alquiler.
*   Evaluar el resultado de la regresión lineal aplicada y compararla con el promediador.

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### Código de descarga del dataset

In [ ]:
if os.access('propiedades.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/propiedades.csv > propiedades.csv
    else:
        !wget propiedades.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/propiedades.csv

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.
df = pd.read_csv("propiedades.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas del DataFrame df
df.head()

In [ ]:
# Eliminar todas las filas de la tabla las cuales el campo "m2" o "ambientes" se encuentre 
#con datos faltantes
# Se hace una copia
df_clean = df.copy()

# Para eliminar los datos faltantes, se implementará el método dropna, indicando 
# La columna a borrar
df_clean.dropna(subset=['m2'], inplace=True)
df_clean.dropna(subset=['ambientes'], inplace=True)

# Otra manera de obtener el mismo resultado:
#df_clean = df[df['m2'].notna()]
#df_clean = df_clean[df_clean['ambientes'].notna()]

In [ ]:
# ¿Cuántos datos quedaron para analizar?
# (filas, columnas)
df_clean.shape

## Fin de la limpieza
Se finalizó la limpieza sacando aquellas filas que no eran de interes, se puede observar que la cantidad de filas hasta el momento es menor a la cantidad de filas con la que se comnezó el análisis

In [ ]:
print('Cantidad de datos en observacion:', df_clean.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Muestra las 5 primeras filas del DataFrame df_clean
df_clean.head()

Se puede observar que hay alquileres en dolares, solo nos quedaremos con aquellos alquileres en pesos

In [ ]:
# Filtramos el dataframe, solos nos quedamos con los alquileres en pesos
# df_ars, almacena el DataFrame filtrado
# Para filtrad el DataFrame llamado df_clean se debe acceder a el con corchetes
# Entre los corcheter especificar la condición de filtrado 
# --> Accede a la columna moneda df_clean['moneda']
# Y para cada valor va a comparar si es igual a 'ARS'

df_ars = df_clean[df_clean['moneda'] == 'ARS']

# shape informa de la cantidad de filas y columnas y en la 
# ubicación 0 está la información de las filas.
print('Cantidad de datos en observacion:', df_ars.shape[0])

In [ ]:
# Renderizamos el gráfico utilizando el diagrama de cajas
#df_ars.boxplot(column=['m2'])

# Se crea el espacio para dibujar con fig = plt.figure(figsize=(16, 9))
# Se crea el espacio para el gráfico
# sns alias Seaborn
# accede al método boxplot(), donde se le indica los valores a representar en el eje de las x, en este caso, 
# los valores de la columna m2
# ax=ax, es un objeto Axes sobre el que se dibuja el gráfico; de lo contrario, utiliza los ejes actuales.
# ax.grid('dashed') para ver la grilla de fondo especificando el color

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df_ars['m2'], ax=ax, color='darkcyan')
ax.grid('dashed')

Se puede observar que hay muchos alquileres "outliers", es decir que se escapan de la tendencia. Filtraremos aquellos que no aportan al análisis

In [ ]:
# Nos quedamos solamente con los deparamentos menor a 200m2 y mayor a 50m2
# Utilizamos el concepto de "máscara" tal cual fue visto en Numpy
# para filtrar el dataframe de propiedads deseado
propiedades = df_ars[(df_ars['m2'] < 200) & (df_ars['m2'] > 50)]

In [ ]:
print('Cantidad de datos en observacion:', propiedades.shape[0])

In [ ]:
# Graficar la tendencia
sns.scatterplot(x=propiedades['m2'], y=propiedades['precio'], color='b', label='precio vs m2')
plt.show()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
# Para los valores de X se emplea doble corchete porque necesita la matriz en dos dimensiones
X = propiedades[['m2']].values

# Para los valores de y se emplea un corchete porque necesita la matriz en una dimensiones
y = propiedades['precio'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
# Se importa la herramienta de sklearn.model_selectionl como train_test_split
from sklearn.model_selection import train_test_split

# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
# Ojo! Los dataset de train y test son array numpy
# Se importa la herramienta de la libreria  train_test_split()
# Necesita los valores de X e y
# test_size=0.3, permite indicar el porcentaje de valores para evaluar, equivalente a un 30%
# random_state=42,  es un número fijo que utilizan comunmente en documentación, significa que para cada ejecución del algoritmo se genere nuevos valores aleatorios
# y los conjuntos de datos de entrenamiento y pruebas serán diferentes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Ahora generaremos un modelo "base", el cual podremos utilizarlo para evaluar en la próxima instancia contra el modelo que construyamos con inteligencia artificial

In [ ]:
# Modelo base simple
# precio_m2 calcula el promedio 
# precio_m2, almacena el resultado de la division de cada y_train y X_train
# promedio_precio_m2, almacena el promedio de los resultados de las divisiones
# Para calcular un precio de alquiler se multiplica los m2 por el precio promedio de alquiler

precio_m2 = y_train / X_train
promedio_precio_m2 = precio_m2.mean()
y_hat_base = X_test * promedio_precio_m2
print(f"Precio del m2 promedio: ${promedio_precio_m2:.2f}")

In [ ]:
# Gráfico con Matplotlib
plt.scatter(X_test, y_hat_base, color='darkGreen', label='y_hat_model')

# Mismo gráfico con Seaborn
# X_test[:,0], de los valores de X_test que grafique todos los valores (:) de la y el 0 indica que todos esten en una dimensión 
#sns.scatterplot(x=X_test[:,0], y=y_hat_base[:,0], color='darkGreen', label='y_hat_model')

# Mostrar el gráfico
plt.show()

In [ ]:
# Se crea el espacio para dibujar con fig = plt.figure()
# Se crea el espacio para el gráfico ax = fig.add_subplot()
# Del DataFrame propiedades accedemos al método plot.scatter para representar m2 y precio reales en un gráfico de dispersión
# De ax accedemos al método scatter para mostrar los m2 y los precios resultados de la predicción del promediador.
fig = plt.figure()
ax = fig.add_subplot()
propiedades.plot.scatter(['m2'], ['precio'], color='b', ax=ax)
ax.scatter(X_test, y_hat_base, color='darkGreen', label="base model")
ax.legend()
plt.show()

Como se observa en el último gráfico, la línea verde aproxima bastante a la distribución de puntos azules, pero debería "apuntar" un poco más arriba. En el próximo encuentro discutiremos porque sucede este fenomeno

In [ ]:
# Calcular los errores del modelo base
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print('Error promedio   MAE - Modelo Base:', mean_absolute_error(y_test, y_hat_base))
print('Error cuadrático MSE - Modelo Base:', mean_squared_error(y_test, y_hat_base))

Sin utilizar inteligencia artificial, unicamente con nuestro conocimientos sobre los datos y estadística hemos generado un primer modelo base competirá contra los algoritmos de inteligencia artificial que veremos en la siguiente etapa

#### Crear un modelo de regresión lineal simple para constrastar contra el modelo base

In [ ]:
# Se importa la herramienta de sklearn.linear_model como LinearRegression
from sklearn.linear_model import LinearRegression

# Se crea el objeto lr a partir que significa Regresión lineal a partir de la clase LinearRegression()
lr = LinearRegression()

# Del objeto lr se puede acceder al método fit con la notación del punto
# Necesita los valores de X haciendole un ajuste con reshape para que haga el entrenamiento junto a los 
# valores de y
lr.fit(X_train, y_train)

# Luego del objeto lr se puede acceder al método predict() que se encarga de hacer las precciones para cada
# valor de X
y_hat= lr.predict(X_test)

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Es un coeficiente de determinación, determina la capacidad de un modelo para predecir futuros resultados. 
# El mejor resultado posible es 1.0
# Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# Se importa la herramienta de sklearn.metrics como r2_score
from sklearn.metrics import r2_score

# En este caso, ya que se trata de una regresión lineal puedo relacinonar el
# índice R2 con el coeficiente de correlación lineal.
value = r2_score(y_test, y_hat)
print(f"Coeficiente de determinación: {value:.2f}")

Se obtuvo un índice de correlación del 40%, parece bajo, ¿será mejor el modelo base?

In [ ]:
value = r2_score(y_test, y_hat_base)
print(f"Coeficiente de determinación: {value:.2f}")

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Modelo de Regresión Lineal
m2 = np.array([100])
y_hat_prueba = lr.predict(m2.reshape(-1, 1))
y_hat_prueba

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Con esto concluye el trabajo que se realizará sobre el dataset de "propiedades". El mejor resultado que se obtuvo fue un ajuste del 40% utilizando una regresión lineal simple.<br>
Aunque ese resultado supera al modelo base, se puede seguir trabajando agregando más entradas al modelo (multilineal) o mejorando la clasificación de propiedades antes de someterlas a la regresión lineal.<br>
Queda en la voluntad del alumno seguir trabajando, puede ser parte de su proyecto final.

#### Ejemplo de aplicación basada en este notebook

In [ ]:
# Visitar la siguiente URL
#http://inove.pythonanywhere.com/machinelearning/alquileres

In [ ]:
# Github de referencia
#https://github.com/InoveProyectos/machine_learning/tree/main/alquileres_regression